<p align="center">
  <span style="font-size:22px"><b>ISEL</b></span><br>
  <span style="font-size:22px"><b>Departamento de Engenharia Informática</b></span><br>
  <span style="font-size:22px"><b>Licenciatura em Engenharia Informática e Multimédia</b></span><br>
  <span style="font-size:22px"><b>Processamento de Sinais Multimédia</b></span><br><br>
  <span style="font-size:18px"><b>1º Trabalho Prático</b></span><br>
  <span style="font-size:16px">3º Semestre 2025/2026</span><br><br>
  <span style="font-size:16px">Docente: Prof. Tiago Gonçalves</span><br>
  <span style="font-size:16px">Docente: Prof. Joel Paulo</span><br>
  <span style="font-size:16px">Data: Outobro</span><br><br>
  <span style="font-size:16px">Trabalho realizado por:</span><br>
  <span style="font-size:16px">David Santos nº51417</span><br>
  <span style="font-size:16px">Bernardo Aguiar nº52483</span><br>
  <span style="font-size:16px">Diogo Costa nº52453</span><br>
</p>